<a href="https://colab.research.google.com/github/mrRobot-95/Voice-classification-with-machine-learn/blob/main/Voice_analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math, random
import torch
from torchaudio import transforms
from tqdm import tqdm
import pandas as pd
from os import walk

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/voice analysis"

voice


In [ ]:
mypath1 = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_SPEECH_COUNTING"
mypath2 = "/content/drive/MyDrive/voice analysis/voice/POSITIVE_SPEECH_COUNTING"
mypath3 = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_COUGH"
mypath4 = "/content/drive/MyDrive/voice analysis/voice/POSITIVE_COUGH"
mypath5 = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_SPEECH_VOWELS"
mypath6 = "/content/drive/MyDrive/voice analysis/voice/POSITIVE_SPEECH_VOWELS"
mypath7 = "/content/drive/MyDrive/voice analysis/voice/BREATHING_HEALTHY"
mypath8 = "/content/drive/MyDrive/voice analysis/voice/BREATHING_POSITIVE"

In [ ]:
x = [mypath1,mypath2]
f = []
for i in x:
    for (dirpath, dirnames, filenames) in walk(i):
        f.extend(filenames)
        break

In [ ]:
d = {}
label = ''
path = ""
file = ""

for i in f:
    if 'HSCN' in i:
        label = 1
        path = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_SPEECH_COUNTING"
        file = path + '/' + i
        d[i] = file, label
    elif 'HSCF' in i:
        label = 2
        path = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_SPEECH_COUNTING"
        file = path + '/' + i
        d[i] = file, label
    elif 'PSCN' in i:
        label = 3
        path = "/content/drive/MyDrive/voice analysis/voice/POSITIVE_SPEECH_COUNTING"
        file = path + '/' + i
        d[i] = file, label
    elif 'PSCF' in i:
        label = 4
        path = "/content/drive/MyDrive/voice analysis/voice/POSITIVE_SPEECH_COUNTING"
        file = path + '/' + i
        d[i] = file, label
    elif 'HCH' in i:
        label = 5
        path = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_COUGH"
        file = path + '/' + i
        d[i] = file, label
    elif 'HCS' in i:
        label = 6
        path = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_COUGH"
        file = path + '/' + i
        d[i] = file, label
    elif 'PCH' in i:
        label = 7
        path = "./voice/POSITIVE_COUGH"
        file = path + '/' + i
        d[i] = file, label
    elif 'PCS' in i:
        label = 8
        path = "/content/drive/MyDrive/voice analysis/voice/POSITIVE_COUGH"
        file = path + '/' + i
        d[i] = file, label
    elif 'HSVA' in i:
        label = 9
        path = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_SPEECH_VOWELS"
        file = path + '/' + i
        d[i] = file, label
    elif 'HSVE' in i:
        label = 10
        path = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_SPEECH_VOWELS"
        file = path + '/' + i
        d[i] = file, label
    elif 'HSVO' in i:
        label = 11
        path = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_SPEECH_VOWELS"
        file = path + '/' + i
        d[i] = file, label
    elif 'PSVA' in i:
        label = 12
        path = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_SPEECH_VOWELS"
        file = path + '/' + i
        d[i] = file, label
    elif 'PSVE' in i:
        label = 13
        path = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_SPEECH_VOWELS"
        file = path + '/' + i
        d[i] = file, label
    elif 'PSVO' in i:
        label = 14
        path = "/content/drive/MyDrive/voice analysis/voice/HEALTHY_SPEECH_VOWELS"
        file = path + '/' + i
        d[i] = file, label
    elif 'HBD' in i:
        label = 15
        path = "/content/drive/MyDrive/voice analysis/voice/BREATHING_HEALTHY"
        file = path + '/' + i
        d[i] = file, label
    elif 'HBS' in i:
        label = 16
        path = "/content/drive/MyDrive/voice analysis/voice/BREATHING_HEALTHY"
        file = path + '/' + i
        d[i] = file, label
    elif 'PBD' in i:
        label = 17
        path = "/content/drive/MyDrive/voice analysis/voice/BREATHING_POSITIVE"
        file = path + '/' + i
        d[i] = file, label
    elif 'PBS' in i:
        label = 18
        path = "/content/drive/MyDrive/voice analysis/voice/BREATHING_POSITIVE"
        file = path + '/' + i
        d[i] = file, label

    d[i] = file, label

df = pd.DataFrame.from_dict(d, orient='index', columns=['relative_path', 'classID'])
print(df.head())

                                                relative_path  classID
53HSCF.wav  /content/drive/MyDrive/voice analysis/voice/HE...        2
53HSCN.wav  /content/drive/MyDrive/voice analysis/voice/HE...        1
54HSCF.wav  /content/drive/MyDrive/voice analysis/voice/HE...        2
54HSCN.wav  /content/drive/MyDrive/voice analysis/voice/HE...        1
55HSCF.wav  /content/drive/MyDrive/voice analysis/voice/HE...        2


In [ ]:
class AudioUtil():
    # ----------------------------
    # Load an audio file. Return the signal as a tensor and the sample rate
    # ----------------------------
    @staticmethod
    def open(audio_file):
        sig, sr = torchaudio.load(audio_file)
        return (sig, sr)

        # ----------------------------
        # Convert the given audio to the desired number of channels
        # ----------------------------

    @staticmethod
    def rechannel(aud, new_channel):
        sig, sr = aud

        if (sig.shape[0] == new_channel):
            # Nothing to do
            return aud

        if (new_channel == 1):
            # Convert from stereo to mono by selecting only the first channel
            resig = sig[:1, :]
        else:
            # Convert from mono to stereo by duplicating the first channel
            resig = torch.cat([sig, sig])

        return ((resig, sr))

        # ----------------------------
        # Since Resample applies to a single channel, we resample one channel at a time
        # ----------------------------

    @staticmethod
    def resample(aud, newsr):
        sig, sr = aud

        if (sr == newsr):
            # Nothing to do
            return aud

        num_channels = sig.shape[0]
        # Resample first channel
        resig = torchaudio.transforms.Resample(sr, newsr)(sig[:1, :])
        if (num_channels > 1):
            # Resample the second channel and merge both channels
            retwo = torchaudio.transforms.Resample(sr, newsr)(sig[1:, :])
            resig = torch.cat([resig, retwo])

        return ((resig, newsr))
        # ----------------------------
        # Shifts the signal to the left or right by some percent. Values at the end
        # are 'wrapped around' to the start of the transformed signal.
        # ----------------------------
        # ----------------------------
        # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds
        # ----------------------------

    @staticmethod
    def pad_trunc(aud, max_ms):
        sig, sr = aud
        num_rows, sig_len = sig.shape
        max_len = sr // 1000 * max_ms

        if (sig_len > max_len):
            # Truncate the signal to the given length
            sig = sig[:, :max_len]

        elif (sig_len < max_len):
            # Length of padding to add at the beginning and end of the signal
            pad_begin_len = random.randint(0, max_len - sig_len)
            pad_end_len = max_len - sig_len - pad_begin_len

            # Pad with 0s
            pad_begin = torch.zeros((num_rows, pad_begin_len))
            pad_end = torch.zeros((num_rows, pad_end_len))

            sig = torch.cat((pad_begin, sig, pad_end), 1)

        return (sig, sr)

    @staticmethod
    def time_shift(aud, shift_limit):
        sig, sr = aud
        _, sig_len = sig.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return (sig.roll(shift_amt), sr)

        # ----------------------------
        # Generate a Spectrogram
        # ----------------------------

    @staticmethod
    def spectro_gram(aud, n_mels=64, n_fft=1024, hop_len=None):
        sig, sr = aud
        top_db = 80

        # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
        spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(sig)

        # Convert to decibels
        spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
        return (spec)

    # ----------------------------
    # Augment the Spectrogram by masking out some sections of it in both the frequency
    # dimension (ie. horizontal bars) and the time dimension (vertical bars) to prevent
    # overfitting and to help the model generalise better. The masked sections are
    # replaced with the mean value.
    # ----------------------------
    @staticmethod
    def spectro_augment(spec, max_mask_pct=0.1, n_freq_masks=1, n_time_masks=1):
        _, n_mels, n_steps = spec.shape
        mask_value = spec.mean()
        aug_spec = spec

        freq_mask_param = max_mask_pct * n_mels
        for _ in range(n_freq_masks):
            aug_spec = transforms.FrequencyMasking(freq_mask_param)(aug_spec, mask_value)

        time_mask_param = max_mask_pct * n_steps
        for _ in range(n_time_masks):
            aug_spec = transforms.TimeMasking(time_mask_param)(aug_spec, mask_value)

        return aug_spec

In [ ]:
from torch.utils.data import DataLoader, Dataset, random_split
import torchaudio


# ----------------------------
# Sound Dataset
# ----------------------------
class SoundDS(Dataset):
    def __init__(self, df, data_path):
        self.df = df
        self.data_path = str(data_path)
        self.duration = 4000
        self.sr = 44100
        self.channel = 2
        self.shift_pct = 0.4

    # ----------------------------
    # Number of items in dataset
    # ----------------------------
    def __len__(self):
        return len(self.df)

        # ----------------------------

    # Get i'th item in dataset
    # ----------------------------
    def __getitem__(self, idx):
        aug_sgram=''
        class_id=''

        # # Absolute file path of the audio file - concatenate the audio directory with
        # # the relative path
        # audio_file = self.df.loc[idx,'relative_path']
        # # Get the Class ID
        # class_id = self.df.loc[idx,'classID']
        for index_num, row in tqdm(df.iterrows()):
            audio_file = str(row["relative_path"])
            class_id = row["classID"]

            aud = AudioUtil.open(audio_file)
            # Some sounds have a higher sample rate, or fewer channels compared to the
            # majority. So make all sounds have the same number of channels and same
            # sample rate. Unless the sample rate is the same, the pad_trunc will still
            # result in arrays of different lengths, even though the sound duration is
            # the same.
            reaud = AudioUtil.resample(aud, self.sr)
            rechan = AudioUtil.rechannel(reaud, self.channel)

            dur_aud = AudioUtil.pad_trunc(rechan, self.duration)
            shift_aud = AudioUtil.time_shift(dur_aud, self.shift_pct)
            sgram = AudioUtil.spectro_gram(shift_aud, n_mels=64, n_fft=1024, hop_len=None)
            aug_sgram = AudioUtil.spectro_augment(sgram, max_mask_pct=0.1, n_freq_masks=2, n_time_masks=2)

        return aug_sgram, class_id

data_path=''

myds = SoundDS(df,data_path)

# Random split of 80:20 between training and validation
num_items = len(myds)
num_train = round(num_items * 0.8)
num_val = num_items - num_train
train_ds, val_ds = random_split(myds, [num_train, num_val])


# Create training and validation data loaders
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=16, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=16, shuffle=False)


In [ ]:
import torch.nn as nn
from torch.nn import init


# ----------------------------
# Audio Classification Model
# ----------------------------
class AudioClassifier(nn.Module):
    # ----------------------------
    # Build the model architecture
    # ----------------------------
    def __init__(self):
        super().__init__()
        conv_layers = []

        # First Convolution Block with Relu and Batch Norm. Use Kaiming Initialization
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]


        # Second Convolution Block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        # Second Convolution Block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.bn3 = nn.BatchNorm2d(32)
        init.kaiming_normal_(self.conv3.weight, a=0.1)
        self.conv3.bias.data.zero_()
        conv_layers += [self.conv3, self.relu3, self.bn3]

        # Second Convolution Block
        self.conv4 = nn.Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu4 = nn.ReLU()
        self.bn4 = nn.BatchNorm2d(64)
        init.kaiming_normal_(self.conv4.weight, a=0.1)
        self.conv4.bias.data.zero_()
        conv_layers += [self.conv4, self.relu4, self.bn4]

        # Linear Classifier
        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=10)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)

    # ----------------------------
    # Forward pass computations
    # ----------------------------
    def forward(self, x):
        # Run the convolutional blocks
        x = self.conv(x)

        # Adaptive pool and flatten for input to linear layer
        x = self.ap(x)
        x = x.view(x.shape[0], -1)

        # Linear layer
        x = self.lin(x)

        # Final output
        return x


# Create the model and put it on the GPU if available
myModel = AudioClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
# Check that it is on Cuda
next(myModel.parameters()).device

device(type='cpu')

In [ ]:

# ----------------------------
# Training Loop
# ----------------------------
def training(model, train_dl, num_epochs):
    # Loss Function, Optimizer and Scheduler
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    print(int(len(train_dl)))
    print(int(len(train_dl))*int(num_epochs))

    print(100)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,
                                                    steps_per_epoch=int(len(train_dl)),
                                                    epochs = num_epochs,
                                                    anneal_strategy='linear')
    print(101)

    # Repeat for each epoch
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct_prediction = 0
        total_prediction = 0
        print(102)

        # Repeat for each batch in the training set
        for i, data in enumerate(train_dl):
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)
            print(104)
            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()
            print(103)

            # Keep stats for Loss and Accuracy
            running_loss += loss.item()

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs, 1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

            if i % 10 == 0:    # print every 10 mini-batches
               print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 10))

        # Print stats at the end of the epoch
        num_batches = len(train_dl)
        avg_loss = running_loss / num_batches
        acc = correct_prediction / total_prediction
        print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

    print('Finished Training')
print(104)
num_epochs = 1  # Just for demo, adjust this higher.
training(myModel, train_dl, num_epochs)
print(105)



# ----------------------------
# Inference
# ----------------------------
def inference(model, val_dl):
    correct_prediction = 0
    total_prediction = 0

    # Disable gradient updates
    with torch.no_grad():
        for data in val_dl:
            # Get the input features and target labels, and put them on the GPU
            inputs, labels = data[0].to(device), data[1].to(device)

            # Normalize the inputs
            inputs_m, inputs_s = inputs.mean(), inputs.std()
            inputs = (inputs - inputs_m) / inputs_s

            # Get predictions
            outputs = model(inputs)

            # Get the predicted class with the highest score
            _, prediction = torch.max(outputs, 1)
            # Count of predictions that matched the target label
            correct_prediction += (prediction == labels).sum().item()
            total_prediction += prediction.shape[0]

    acc = correct_prediction / total_prediction
    print(f'Accuracy: {acc:.2f}, Total items: {total_prediction}')


# Run inference on trained model with the validation set
inference(myModel, val_dl)

104
58
58
100
101
102


1157it [00:52, 22.25it/s]
1157it [00:49, 23.24it/s]
1157it [00:50, 23.02it/s]
1157it [00:48, 23.68it/s]
1157it [00:49, 23.58it/s]
1157it [00:50, 23.01it/s]
1157it [00:49, 23.32it/s]
1157it [00:57, 20.28it/s]
1157it [00:50, 23.09it/s]
1157it [00:49, 23.36it/s]
1157it [00:50, 22.85it/s]
1157it [00:50, 23.00it/s]
1157it [00:51, 22.39it/s]
1157it [00:49, 23.50it/s]
1157it [00:48, 23.70it/s]
1157it [00:51, 22.55it/s]


104
103
[1,     1] loss: 0.217


1157it [00:51, 22.68it/s]
1157it [00:50, 23.03it/s]
1157it [00:50, 22.72it/s]
1157it [00:51, 22.29it/s]
1157it [00:51, 22.64it/s]
1157it [00:52, 22.24it/s]
1157it [00:51, 22.57it/s]
1157it [00:52, 21.86it/s]
1157it [00:51, 22.60it/s]
1157it [00:52, 22.22it/s]
1157it [00:51, 22.55it/s]
1157it [00:50, 22.77it/s]
1157it [00:52, 22.10it/s]
1157it [00:49, 23.19it/s]
1157it [00:52, 22.11it/s]
1157it [00:52, 22.09it/s]


104
103


...

1157it [01:22, 13.94it/s]
1157it [00:58, 19.85it/s]
1157it [00:54, 21.10it/s]
1157it [00:52, 21.86it/s]
1157it [00:50, 23.04it/s]
1157it [00:50, 22.77it/s]
1157it [00:49, 23.33it/s]
1157it [00:50, 22.93it/s]
1157it [00:49, 23.26it/s]
1157it [00:49, 23.34it/s]
1157it [00:50, 22.88it/s]
1157it [00:49, 23.36it/s]
1157it [00:49, 23.32it/s]
1157it [00:50, 22.71it/s]
1157it [00:50, 23.04it/s]
1157it [00:50, 22.82it/s]


104
103


1157it [00:49, 23.14it/s]
1157it [00:50, 22.88it/s]
1157it [00:50, 22.87it/s]
1157it [00:49, 23.22it/s]
1157it [00:49, 23.21it/s]
1157it [00:51, 22.30it/s]
1157it [00:50, 22.89it/s]
1157it [00:51, 22.53it/s]
1157it [00:48, 23.78it/s]
1157it [00:48, 24.10it/s]
1157it [00:50, 23.14it/s]
1157it [00:48, 24.02it/s]
1157it [00:48, 23.87it/s]
1157it [00:49, 23.47it/s]
1157it [00:47, 24.30it/s]
1157it [00:48, 23.76it/s]


104
103


1157it [00:47, 24.42it/s]
1157it [00:47, 24.30it/s]
1157it [00:48, 23.99it/s]
1157it [00:47, 24.49it/s]
1157it [00:47, 24.37it/s]
1157it [00:48, 23.95it/s]
1157it [00:47, 24.37it/s]
645it [00:26, 21.04it/s]